<a href="https://colab.research.google.com/github/KOMBOU12/Marius/blob/main/NLP_R%C3%A9sum%C3%A9_d'article.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Voici l'implémentation que j'ai crée qui divise les articles avant de les résumer.

Mais ça prend trop de temps pour être exécuter sur les 400 articles du train set OBS.

Peut-être l'essayer sur 10 articles ne serait pas une mauvaise idée.

Ici le modèle est déjà préentrainé.


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import os
import pandas as pd

# Charger le modèle LED pour résumer des textes longs
model_name = "allenai/led-large-16384"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Dossier contenant les fichiers d'articles
dossier_articles = "/kaggle/input/m-2-maliash-resume-darticles-scientifiques/train/OBS/articles_OBS/"

# Fonction pour lire un fichier
def lire_article(chemin_dossier, nom_fichier):
    chemin_complet = os.path.join(chemin_dossier, nom_fichier)
    with open(chemin_complet, 'r', encoding='utf-8') as fichier:
        return fichier.read()

# Fonction pour résumer un texte long
def resumer_texte_long(texte, max_length=512, chunk_size=4096):
    # Diviser le texte en morceaux
    morceaux = [texte[i:i + chunk_size] for i in range(0, len(texte), chunk_size)]
    resumes = []
    for morceau in morceaux:
        # Générer un résumé pour chaque morceau
        inputs = tokenizer(morceau, return_tensors="pt", truncation=True, max_length=chunk_size)
        outputs = model.generate(inputs["input_ids"], max_length=max_length, min_length=50, length_penalty=2.0)
        resumes.append(tokenizer.decode(outputs[0], skip_special_tokens=True))

    # Combiner les résumés partiels en un seul texte
    texte_final = " ".join(resumes)
    return texte_final

# Lister tous les fichiers d'articles
fichiers_articles = [f for f in os.listdir(dossier_articles) if f.startswith("article-")]

# Liste pour stocker les résumés générés
resumes_generes = []

# Traiter chaque fichier d'article
for article_fichier in fichiers_articles:
    try:
        # Lire le contenu de l'article
        contenu_article = lire_article(dossier_articles, article_fichier)

        # Résumer l'article
        resume = resumer_texte_long(contenu_article, max_length=512, chunk_size=4096)

        # Ajouter le résumé et l'identifiant à la liste
        resumes_generes.append({
            "fichier": article_fichier,
            "resume": resume
        })

        # Afficher un message de progression
        print(f"Résumé généré pour : {article_fichier}")

    except Exception as e:
        print(f"Erreur pour l'article {article_fichier}: {e}")

# Sauvegarder les résumés dans un fichier CSV
df_resumes = pd.DataFrame(resumes_generes)
df_resumes.to_csv("resumes_articles.csv", index=False, encoding="utf-8")
print("Tous les résumés ont été sauvegardés dans 'resumes_articles.csv'")


**Importationn du modèle préentrainé**

In [ ]:
import os
import random
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import evaluate
import gc

hf_token = "" #Mettre son token dans Kaggle

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct", token=hf_token)
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.2-3B-Instruct",
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
    token=hf_token
)

**Partie du code pour résumer 10 articles du dossier test et pour obtenir les score**

In [ ]:
import os
import random
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import evaluate
import gc



articles_dir = "/kaggle/input/m-2-maliash-resume-darticles-scientifiques/test/OBS_test/articles_OBS_test/"
article_files = [f for f in os.listdir(articles_dir) if f.endswith('.txt')]
selected_articles = random.sample(article_files, 10)

summaries = []
rouge = evaluate.load("rouge")

for article_file in selected_articles:
    article_path = os.path.join(articles_dir, article_file)
    with open(article_path, "r", encoding="utf-8") as file:
        article_text = file.read()


    prompt =[
    {"role":"system", "content" : "You are an expert assistant specialized in summarizing scientific articles. Provide a structured summary"},
     {"role":"user", "content" : article_text}]
    chat = tokenizer.apply_chat_template(prompt, tokenize=False)

    input_ids = tokenizer(chat, add_special_tokens=True, return_tensors="pt").to("cuda")

    try:
        output_ids = model.generate(
            **input_ids, #input_ids["input_ids"],
            #attention_mask=input_ids["attention_mask"],
            do_sample=True,
            top_p=50,  # Réduction pour privilégier les options les plus probables
            temperature=1,  # Réduction pour limiter la stochasticité
            max_new_tokens=150  # Plus de tokens pour des résumés plus longs
        )
        summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    except torch.cuda.OutOfMemoryError:
        print(f"Erreur de mémoire pour {article_file}. Réduction des tokens...")
        torch.cuda.empty_cache()
        gc.collect()
        continue

    summaries.append({"Article_ID": article_file, "Summary": summary})

    # Remplacez par un vrai résumé de référence
    reference_summary = "Résumé réel de l'article ici."
    results = rouge.compute(predictions=[summary], references=[reference_summary])

    print(f"Scores ROUGE pour {article_file}:")
    for key, value in results.items():
        print(f"  {key}: {value}")

    torch.cuda.empty_cache()
    gc.collect()


submission_path = "/kaggle/working/submission2.csv"
submission2 = pd.DataFrame(summaries)
submission2.to_csv(submission_path, index=False)

Il faut chercher à améliorer les scores....